# Постановка задачи

Найдите решение следующей задачи с помощью генетического алгоритма:
$$
y''(x) + \frac{1}{x}y'(x) = - \frac{x^2(1+x)}{x+3}, \;\ x \in [0, \pi],\\
y(0)= 0, \;\ y(\pi) = 0.
$$
Немного преобразуем систему
$$
xy''(x) + y'(x) = - \frac{x^3(1+x)}{x+3}, \;\ x \in [0, \pi],\\
y(0)= 0, \;\ y(\pi) = 0.
$$


# Аналитическое решение

Пусть $y'(x) = z(x).$ Тогда можно получить решение $z(x):$

$$
x(x+3)z'(x) + (x+3)z(x) = -x^3(1+x),\\
z(x) = \frac{c_1}{x} - \frac{x^3}{4} + \frac{2x^2}{3} - 3x + 18 - \frac{54}{x} ln(x+3)
$$

Таким образом необходимо вычислить $y(x) = \int z(x) dx$
$$
y(x) = c_2 + c_1ln(x) - \frac{x^4}{16} + \frac{2x^3}{9} - \frac{3x^2}{2} + 18x - \int \frac{54}{x} ln(x+3)dx
$$

[Проверить можно здесь](https://www.wolframalpha.com/input/?i=x*y%27%27%28x%29+%2B+y%27%28x%29+%3D+-+x%5E3%281%2Bx%29%2F%28x%2B3%29)

# Алгоритм численного решения
Рассмотрим функционал
$$
V(y) = ||Ay - f||^2 = || xy''(x) + y'(x) + \frac{x^3(1+x)}{x+3}||^2 \sim \min_{y}
$$

Введем функцию приспособления $\displaystyle F(y) = \frac{1}{1+V(y)} \in [0,1]$

## Начальная популяция
Начальную популяцию будем генерировать следующим образом:
$$
y_n = \sum_{m=1}^{M} C_m^i f(x,m), \;\ n = 1, ..., N,
$$
где $C_m^i$ и $M$ выбираются случайно. В качестве базисный функций $f(x,m)$ можно взять,например, $sin(mx)$ или $x^m(x-\pi)^m.$

## Отбор

Выберем 
$$y^{n+1}_1 = y^{n}_0: \max_{i} F(y^{n}_i)$$

Выберем порог выживаемости $s$, например,$s = \frac{3}{4}.$ Далее выберем $sN$ функций турнирным отбором:
\begin{equation*}
y^{n+1}_i = 
 \begin{cases}
   y^{n+1}_j, & F(y^{n+1}_j) > F(y^{n+1}_k), \\
   y^{n+1}_k, & F(y^{n+1}_j) \leq F(y^{n+1}_k), \;\ i = 2,..., \frac{3}{4}N.
 \end{cases}
\end{equation*}
где $j,k$ выбраны случайно. Популяция $[sN, N]$ заполняется случайным образом.

Вместо этого можно, например, отранжировать популяцию по значению функции приспособленности и заново сгенерировать $[sN, N]$ функции.

## Выбор родителей

Составляем случайно $N$ пар вида:
$$
y_j^{n+1}, P_j = \frac{F(y_j^{n+1})}{\sum_{i=1}^{N} F(y_i^{n+1})}, \\
y_k^{n+1}, P_k = \frac{F(y_k^{n+1})}{\sum_{i=1}^{N} F(y_i^{n+1})},
$$
где $P_i$ - вероятность выбора $i$-ой функции в качестве родителя. Будем составлять эти пары так, чтобы родители не повторялись

## Скрещивание:
Данным ниже способ скрещивания показался не очень эффективным:
$$
y_i^{n+1} = \frac{F(y_j^{n+1})y_j^{n+1} + F(y_k^{n+1}) y_k^{n+1}}{F(y_j^{n+1}) + F(y_k^{n+1})}
$$

Поэтому был выбран метод промежуточной рекомбинации:
$$
y_i^{n+1} = y_j^{n+1} - \alpha_i(y_k^{n+1} - y_j^{n+1}),
$$
где $\alpha_i$ - коэффициент промежуточной рекомбинации.

# Main

In [27]:
import numpy as np
from math import pi
import sympy as sp
import random

In [28]:
N = 50 # Число "особей" в популяции
a, b = 0, pi # Границы переменной x 

x = sp.symbols('x')

In [41]:
class Genetic:
    C = sp.IndexedBase('C')
    
    def __init__(self, N:int, M:int, task=0, mode=0):
        # Размер популяции
        self.N = N
        
        # Выберем задачу и вид базисных функций
        self.task = task # 0 - тест, 1 - поставленная задача
        self.mode = mode # 0 - sin(mx), 1 - x**m*(x-pi)**m, 2 - x**m
        
        # Число базисных функций
        self.M = M
        # Базисные функции
        self.base_functions(self.M)
        
        # Начальная популяция
        self.population = self.get_init_population(N)
        
        
    def base_functions(self, M):
        """ Инициализация базисных функций и функционала"""
        # Инициализируем символьные переменные
        x, m = sp.symbols('x m')
        
        # Выберем базисные функции
        if self.mode == 0: # sin(m*x)
            # Линейная комбинация базисных функций
            s = sp.Sum(self.C[m]*sp.sin(m*x),(m,0,self.M-1)).doit()
            self.base = lambda C: s.subs(C)
        elif self.mode == 1: # x**m*(x-pi)**m
            # Линейная комбинация базисных функций
            s = sp.Sum(self.C[m]*x**m*(x-pi)**m,(m,0,self.M-1)).doit()
            self.base = sp.lambdify(self.C, s)
        elif self.mode == 2: # x**m
            # Линейная комбинация базисных функций
            s = sp.Sum(self.C[m]*x**m,(m,0,self.M-1)).doit()
            self.base = sp.lambdify(self.C, s)
        
            
        # Функционал, который минимизируем
        if self.task == 0:
            V = sp.diff(s,x,2) + 2*sp.sin(x)
        elif self.task == 1:
            #V = (x+3)*x*sp.diff(s,x,2) + (x+3)*sp.diff(s,x) + x**3*(1+x)
            V = x*sp.diff(s,x,2) + sp.diff(s,x) + x**3*(1+x)/(x+3)
        self.V = sp.lambdify([self.C, x], V)
        
    
    def fitness(self, C, a, b, num=100):
        """ Вычисление функции приспособления для пробной функции"""
        # Вычислим норму функционала
        V = np.linalg.norm(
                    self.V(C, np.linspace(a, b, num)),
                    axis=-1
        ) ** 2
        
        # Значение функции приспособления
        return 1/(1 + V)
        
    def get_init_population(self, N:int):
        """ Получение начальной популяции"""
        # Инициализируем случайное число базисных функций для каждой пробной функции
        M = np.random.randint(2, self.M, N)
        # Пробные функции будем идентифицировать коэффициентами при базисных функциях
        Cm = np.random.uniform(-2, 2, (N, self.M))
        
        # Инициализируем пустой массив для хранения пробных фугкций и значений функции приспособления
        y = np.array([(None, None) for _ in range(N)])
        
        for n in range(N):
            # Пробная функция
            y[n,0] = np.array([Cm[n,m] for m in range(0,M[n])] + [0 for m in range(M[n], self.M)])
            # Значение ее функции приспособленности
            y[n,1] = self.fitness(y[n,0], a, b)
        return y

    
    def selection(self, s=1/4):
        """ Отбор функций производится по рейтингу 
        на основании значения функции приспособления
        """
        # Отсортируем по функции приспособления
        self.population = self.population[self.population[:,1].argsort()[::-1]]

        # Число "выживших особей"
        threshold = int(s*N)
        
        # Восстанавливаем популяцию
        self.population[threshold:self.N] = self.get_init_population(self.N-threshold)
        
    
    def crossing(self, d=1/4):
        """ Скрещивание особей в популяции"""
        # Отсортируем по функции приспособления
        self.population = self.population[self.population[:,1].argsort()[::-1]]
        
        # Вероятность особи стать родителем
        tourney_list = self.population[:,1]
        perm = list(tourney_list/np.linalg.norm(tourney_list, ord = 1))
        
        # Сохраним старую популяцию для скрещивания
        y = self.population[:,0].copy()
        
        # Массив с номерами выбранных индексов
        s = set()
        # Очищаем множества от одноэлементных
        bad_sets = {frozenset([n]) for n in range(self.N)}
        # Пока не получим нужное число пар родителей
        while len(s) < self.N:
            # Выбираем согласно вероятности стать родителем 2N пар особей
            s1 = np.random.choice(self.N, (2*self.N,2), p=perm)
            # Убираем пары в которых только один родитель
            s1 = set(map(frozenset,s1)) - bad_sets
            s.update(s1)
        
        # Коэффициент промежуточной рекомбинации
        alpha = (1 + d + d) * np.random.random(self.N) - d    
        
        for n, (j, k) in zip(range(self.N), s):
            #y_new[n,0] = (tourney_list[j]*y[j] + tourney_list[k]*y[k])/(tourney_list[j] + tourney_list[k])
            
            # Промежуточная рекомбинация
            self.population[n,0] = y[j] - alpha[n]*(y[k] - y[j])
            # Значение ее функции приспособленности
            self.population[n,1] = self.fitness(self.population[n,0], a, b)
        
    def mutating(self, s=1/2):
        """ Мутирование s доли особей"""
        # Число "мутировавших особей"
        threshold = int(s*self.N)
        
        # Инициализируем случайное число базисных функций для каждой мутации
        M = np.random.randint(2, self.M, threshold)
        # Коэффициенты при базисных функция для каждой мутации
        Cm = np.random.uniform(-0.1, 0.1, (threshold, self.M))
        
        # Для случайности перемешаем популяцию
        np.random.shuffle(gen.population)
        for n in range(threshold):
            # Мутация особи
            self.population[n,0] += np.array([Cm[n,m] for m in range(0,M[n])] + [0 for m in range(M[n], self.M)])
            # Значение ее функции приспособленности
            self.population[n,1] = self.fitness(self.population[n,0], a, b)

# Тестовый пример:

Рассмотрим тест:
$$ y''(x) = -2sin(x) \\
y(0)= 0, \;\ y(\pi) = 0
$$

Его решение $\displaystyle y(x) = 2sin(x).$ Выберем базисные функции в виде $sin(mx)$

In [43]:
gen = Genetic(N, 7, 0, 0)

for k in range(10):
    # Отбор
    gen.selection()
    
    # Выведем значение функции приспособленности для лучшей особи
    print(gen.population[0,1])
    
    # Выбор родителей и скрещивание
    gen.crossing()
    
    # Мутация
    gen.mutating()

0.9986740897183201
0.9951419908763073
0.9996328592876347
0.9950593029447826
0.9991962630274697
0.9996128676802987
0.9997114864315789
0.9998759354180831
0.9999340245776928
0.9999387358224304


Выведем коэффициенты наилучшей из отобранных функций:
$$C_0sin(0*x) + C_1sin(1*x) + C_2sin(2*x) + ... $$

In [44]:
gen.selection()
replacements = {gen.C[m]: gen.population[0,0][m] for m in range(len(gen.population[0,0]))}
best = gen.base(replacements)
print('score:',gen.population[0,1])
best

score: 0.9999918953639928


2.00036336522464*sin(x) + 4.45091710038441e-5*sin(2*x)

Рассмотрим второй выбор базисных функций $x^m(x-\pi)^m$. В зависимости от запусков можно преодолеть плато в 0.88

In [37]:
gen = Genetic(N, 7, 0, 1)

for k in range(200):
    # Отбор
    gen.selection()
    
    # Выведем значение функции приспособленности для лучшей особи
    print(gen.population[0,1])
    
    # Выбор родителей и скрещивание
    gen.crossing()
    
    # Мутация
    gen.mutating()

0.024913440319083966
0.024681946166592902
0.39751810245537295
0.706785104310903
0.6513855262219664
0.7846020894698085
0.8587406916249963
0.9106791126104424
0.8911301194127005
0.9001702416786783
0.9224899027012755
0.921629610464577
0.9236955795143078
0.9360618995344625
0.9442210256012283
0.9545377746304967
0.9597624827809688
0.9607803824843775
0.961174316602329
0.9609202071910301
0.9611654133528826
0.9618387296872868
0.9627280776847008
0.9633686226215922
0.9639362848641846
0.9641725504156193
0.9645729869151044
0.9645518320851599
0.9643075684491661
0.964611949710357
0.9685569216565495
0.9699440756818911
0.966514207495305
0.969999425239456
0.9715552486364449
0.972139162936919
0.9733383418277542
0.973122468610671
0.9737864694596166
0.9737880558289447
0.9763278466926039
0.9766406912616284
0.9768725001764009
0.9769518370379432
0.9781497120557805
0.9788337373070678
0.9788841372560529
0.9788653052257081
0.9788590313944914
0.9788655011780573
0.9791423122143863
0.9791208878928073
0.9791968615337

Выведем коэффициенты наилучшей из отобранных функций:
$$C_0x^0 + C_1x^1 + C_2x^2 + C_3x^3 + C_4x^4 ... $$

In [38]:
gen.selection()
best = sp.expand(gen.base(gen.population[0,0]))
print('score:',gen.population[0,1])
best

score: 0.9994721860550421


-0.0024980194556827*x**6 + 0.0235432787114914*x**5 - 0.00975077105737383*x**4 - 0.326005510536985*x**3 - 0.00194091463475499*x**2 + 1.99709183715315*x - 10.4012591751954

# Численный расчет поставленной задачи

- Базисные функции $sin(xm)$ не сходятся за 2к итераций.
- Базисные функции $x^m(x-\pi)^m$ тоже не сходятся за 2к итераций.
- Поэтому расмотрим просто базисные функции $x^m$

В зависимости от случайности значение функции приспособлености может зависнуть на плато в 0.97

In [52]:
gen = Genetic(N, 7, task=1, mode=2)

for k in range(500):
    # Отбор
    gen.selection()
    
    # Выведем значение функции приспособленности для лучшей особи
    print(gen.population[0,1])
    
    # Выбор родителей и скрещивание
    gen.crossing()
    
    # Мутация
    gen.mutating()

0.0006718881890350683
0.0013942321502329737
0.016301455076274222
0.01742477817358137
0.019821192317942393
0.02182012624866727
0.0252297302502828
0.029475700000564646
0.0424397800556895
0.05624463229643877
0.06923984419582267
0.07878917423719242
0.10464404892953999
0.1087213349450683
0.2108510777729234
0.17734035839557127
0.1868063806508516
0.23618204374458918
0.2724432691168389
0.5337541702547646
0.7200592780990686
0.8167013853783559
0.8545365777875856
0.8494691650596826
0.8847977129800099
0.9088214186820065
0.9434106797618415
0.9396220009810051
0.9398214205707442
0.9448446042478251
0.9478370349331673
0.9510282791754167
0.954850139081234
0.9590324102915704
0.9591031079418021
0.9610285257165679
0.9623285675650344
0.961979293841384
0.9637500650401146
0.9646951856954182
0.9665553922850907
0.9684739323544106
0.9697304513606659
0.9705247509865098
0.9708283042657566
0.9711096401429746
0.9711505798458819
0.9712682732952818
0.9712728639696241
0.9713345677210331
0.9714368644302489
0.97158234825

0.9726193804946355
0.9726193804946924
0.9726193804946792
0.9726193804946937
0.9726193804947134
0.9726193804947363
0.9726193804947706
0.9726193804948405
0.9726193804948844
0.9726193804948949
0.9726193804949149
0.9726193804949121
0.9726193804949195
0.9726193804949185
0.9726193804949361
0.9726193804949567
0.972619380494967
0.9726193804949936
0.9726193804950302
0.972619380495084
0.9726193804951409
0.972619380495211
0.9726193804953356
0.9726193804953761
0.9726193804954829
0.972619380495543
0.9726193804955442
0.9726193804955529
0.9726193804955583
0.9726193804955579
0.972619380495561
0.9726193804955654
0.9726193804955708
0.9726193804955762
0.9726193804955786
0.9726193804955849
0.9726193804955887
0.9726193804955942
0.9726193804955977
0.9726193804956028
0.9726193804956091
0.9726193804956108
0.9726193804956125
0.9726193804956151
0.9726193804956207
0.9726193804956234
0.9726193804956252
0.9726193804956271
0.9726193804956299
0.9726193804956307
0.9726193804956349
0.9726193804956372
0.972619380495640

Выведем наилучшую функцию и значение её функции приспособленности (при этом ГУ не удовлетворяются, для этого надо изменить констаты $C_0$ и $C_1$ в соответствии с ГУ)
$$C_0x^0 + C_1x^1 + C_2x^2 + ... $$

In [53]:
gen.selection()
best = sp.expand(gen.base(gen.population[0,0]))
print('score:',gen.population[0,1])
best

score: 0.9726193804956598


-0.0532180100749434*x**4 + 0.0750707737669755*x**3 - 0.0934260722003932*x**2 + 0.0522162726534253*x - 0.325042769143119

Коэффициенты $C_0, C_1, ...$

In [56]:
gen.population[0,0]

array([-0.32504277,  0.05221627, -0.09342607,  0.07507077, -0.05321801,
        0.        ,  0.        ])